In [10]:
import openai
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

In [7]:
client = OpenAI()

In [2]:
df = pd.read_csv("../data/processed/scraped_data_with_LLM_friendly_name.csv")

In [22]:
PROMPT = """You will be given a name of a food item. The item can be of any category of food - from raw ingredients to fully prepared meals.
Your task is to estimate the calories, protein, fat, and carbohydrates per 100 gram of this item.
Firstly, think about the content of the item inside <scratchpad> XML tags.
Then, write the answer while keeping the following structure:
<calories>[Your estimated calories (in grams) per 100 gram]</calories>
<protein>[Your estimated protein (in grams) per 100 gram]</protein>
<fat>[Your estimated fat (in grams) per 100 gram]</fat>
<carbohydrates>[Your estimated carbohydrates (in grams) per 100 gram]</carbohydrates>

To faciliate postprocessing, in each answer section write only a number, without the unit.

Here is the name of the item:
{name}
"""

In [23]:
def ask_gpt(name):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": PROMPT.format(name=name)
            }
        ],
        model="gpt-4o",
    )
    return chat_completion.choices[0].message.content

In [24]:
for idx, row in tqdm(df.iterrows()):
    gpt_estimated_contents = ask_gpt(row["LLM friendly name"])
    df.loc[idx, "gpt_completion"] = gpt_estimated_contents

0it [00:00, ?it/s]

81it [03:40,  2.72s/it]


In [25]:
df['gpt_energy_kcal_100g'] = df['gpt_completion'].apply(lambda x: float(x.split("<calories>")[1].split("</calories>")[0]))
df['gpt_proteins_100g'] = df['gpt_completion'].apply(lambda x: float(x.split("<protein>")[1].split("</protein>")[0]))
df['gpt_fat_100g'] = df['gpt_completion'].apply(lambda x: float(x.split("<fat>")[1].split("</fat>")[0]))
df['gpt_carbohydrates_100g'] = df['gpt_completion'].apply(lambda x: float(x.split("<carbohydrates>")[1].split("</carbohydrates>")[0]))

In [27]:
df.to_csv("../data/processed/scraped_data_with_gpt_estimated_contents.csv", index=False)